In [1]:
import torch 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

%cd ../

/workspace/Walk_Video_PyTorch/project


In [2]:
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()

In [3]:
from pytorch_lightning import seed_everything

seed_everything(42, workers=True)

Global seed set to 42


42

In [4]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 8
opt.batch_size = 8
opt.gpu_num = 0
opt.model = "resnet"
opt.model_depth = 50
opt.version = '0719' + '_' + opt.model + '_depth' + str(opt.model_depth)


In [5]:
from utils.utils import get_ckpt_path

model = WalkVideoClassificationLightningModule(opt)

# get last ckpt path
ckpt_path = get_ckpt_path(opt)


model = model.load_from_checkpoint(ckpt_path)

model.eval()

clear_output()
print(ckpt_path)


/workspace/Walk_Video_PyTorch/logs/resnet/0719_resnet_depth50/checkpoints/epoch=99-step=11200.ckpt


In [6]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)

trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
    max_epochs=1,
    deterministic=True,
)

trainer.test(dataloaders=module, model=model)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. Th

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.3203372061252594     │
│      test_loss_epoch      │    3.3243775367736816     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 3.3243775367736816, 'test_acc_epoch': 0.3203372061252594}]

In [7]:
trainer.validate(model=model, dataloaders=module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 7. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_acc_epoch       │    0.33403581380844116    │
│      val_loss_epoch       │    3.2820286750793457     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss_epoch': 3.2820286750793457, 'val_acc_epoch': 0.33403581380844116}]

In [8]:
input_data = next(iter(test_data))


In [9]:
input_data['video'].shape

input_data["video_name"]

['20200528_DHS_lat_V1-0027.mp4',
 '20200528_DHS_lat_V1-0027.mp4',
 '20200528_DHS_lat_V1-0027.mp4',
 '20200528_DHS_lat_V1-0027.mp4',
 '20210406_ASD_lat_V1-0030.mp4',
 '20200528_DHS_lat_V1-0006.mp4',
 '20200528_DHS_lat_V1-0006.mp4',
 '20200528_DHS_lat_V1-0006.mp4']

In [10]:
input_data['label']

tensor([1, 1, 1, 1, 0, 1, 1, 1])

In [11]:
preds = model(input_data['video'])

In [12]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [13]:
pred_classes

tensor([[0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0]])

In [14]:
from torch import softmax


pred_chagne = softmax(preds, dim=-1)
pred_chagne

tensor([[0.7309, 0.2691],
        [0.7311, 0.2689],
        [0.7311, 0.2689],
        [0.7310, 0.2690],
        [0.2690, 0.7310],
        [0.7310, 0.2690],
        [0.7308, 0.2692],
        [0.7310, 0.2690]], grad_fn=<SoftmaxBackward0>)

In [15]:
from utils.metrics import get_Accuracy

accuracy = get_Accuracy()

accuracy(pred_chagne, input_data['label'])

tensor(0.)

In [16]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [17]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [18]:
real_calss

['asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd',
 'asd_not',
 'asd_not',
 'asd_not']

In [19]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

Predicted labels: asd, asd, asd, asd, asd_not, asd, asd, asd
real label: asd_not,asd_not,asd_not,asd_not,asd,asd_not,asd_not,asd_not


In [20]:
pred_class_names == real_calss

False

In [21]:
result = []

for i in range(len(real_calss)):
    if pred_class_names[i] == real_calss[i]:
        result.append("true")
    else:
        result.append("false")

result

['false', 'false', 'false', 'false', 'false', 'false', 'false', 'false']